In [1]:
import os ,json, logging
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool,StructuredTool
load_dotenv(dotenv_path='.env')
OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')
llm=ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.2
)

In [2]:
@tool
def get_exchange_rate(pair:str)->str:
    """Tra cứu tỷ giá cơ bản (demo). pair ví dụ: 'USD/VND'."""
    rates = {"USD/VND": 24500, "EUR/VND": 26600}
    return str(rates.get(pair.upper(), "Unknown"))


In [3]:
llm_tools=llm.bind_tools([get_exchange_rate])

In [7]:
from langchain_core.messages import ToolMessage

# 1️⃣ Gọi LLM lần đầu
resp = llm_tools.invoke("Hỏi: Bạn khỏe không")

print("Tool calls:", resp.tool_calls)
# resp.content rỗng vì LLM mới yêu cầu gọi tool thôi

# 2️⃣ Nếu có tool_calls thì chạy tool thật
if resp.tool_calls:
    call = resp.tool_calls[0]  # ví dụ chỉ 1 tool
    tool_name = call["name"]
    args = call["args"]
    tool_id = call["id"]

    # Gọi tool thật
    result = get_exchange_rate.invoke(args)

    # Gửi lại kết quả cho LLM qua ToolMessage
    follow_up = llm_tools.invoke([
        resp,  # AIMessage trước đó
        ToolMessage(content=str(result), tool_call_id=tool_id)
    ])

    print("✅ Final answer:", follow_up.content)
else:
    print("✅ Final answer:", resp.content)


Tool calls: []
✅ Final answer: Cảm ơn bạn đã hỏi! Tôi không có cảm xúc, nhưng tôi luôn sẵn sàng giúp đỡ bạn. Bạn cần thông tin gì hôm nay?


In [14]:
args

{'pair': 'USD/VND'}

In [12]:
# =============================
# Phase 2 – Verbose Tool Calling Demo
# =============================
# pip install -U langchain langchain-openai

import os, json, time
from typing import List, Dict, Any
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool

# ====== MÀU in console (ANSI) ======
RESET = "\x1b[0m"
BOLD  = "\x1b[1m"
CYAN  = "\x1b[36m"
GREEN = "\x1b[32m"
YELL  = "\x1b[33m"
MAG   = "\x1b[35m"
RED   = "\x1b[31m"
GRAY  = "\x1b[90m"

def p_sys(s):  print(BOLD + CYAN + s + RESET)
def p_user(s): print(BOLD + GREEN + s + RESET)
def p_ai(s):   print(BOLD + MAG + s + RESET)
def p_tool(s): print(BOLD + YELL + s + RESET)
def p_err(s):  print(BOLD + RED + s + RESET)
def p_dbg(s):  print(GRAY + s + RESET)

# ====== 1) Khởi tạo LLM ======
# Lưu ý: thay model nếu bạn đang dùng model khác
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ====== 2) Định nghĩa tool ======
@tool
def get_exchange_rate(pair: str) -> str:
    """Tra cứu tỷ giá cơ bản (demo). pair ví dụ: 'USD/VND'."""
    rates = {"USD/VND": 24500, "EUR/VND": 26600}
    return str(rates.get(pair.upper(), "Unknown"))

TOOLS = {get_exchange_rate.name: get_exchange_rate}

# ====== 3) Hàm chạy 1 vòng parse->call->final với in màu chi tiết ======
def run_with_tools_verbose(llm, tools, query_or_messages):
    """
    - Nhận string (prompt) hoặc list[BaseMessage]
    - bind_tools -> invoke lần 1
    - nếu có tool_calls: thực thi tool(s) -> ToolMessage -> invoke lần 2
    - In ra từng bước, có màu
    """
    bound = llm.bind_tools(tools)

    # chuẩn hoá input thành danh sách message
    if isinstance(query_or_messages, str):
        messages = [SystemMessage(content="Bạn là trợ lý biết gọi tool chính xác, trả lời ngắn gọn."),
                    HumanMessage(content=query_or_messages)]
    else:
        messages = query_or_messages

    p_sys("=== STEP 1: USER → LLM ===")
    p_user(f"Human: {messages[-1].content}")

    ai1: AIMessage = bound.invoke(messages)
    p_ai(f"AI (lần 1): content = {repr(ai1.content)}")
    p_dbg(f"AI (lần 1): tool_calls = {json.dumps(ai1.tool_calls or [], ensure_ascii=False)}")

    # Nếu không có tool_calls → trả lời luôn
    if not ai1.tool_calls:
        p_ai("→ Không cần tool. Final answer:")
        print(BOLD + ai1.content + RESET)
        return ai1.content

    # Có tool_calls → thực thi từng tool
    tool_msgs: List[ToolMessage] = []
    p_sys("\n=== STEP 2: EXECUTE TOOLS ===")
    for i, call in enumerate(ai1.tool_calls, start=1):
        name = call["name"]
        args = call.get("args", {}) or {}
        t_id = call["id"]

        p_tool(f"[ToolCall {i}] name={name}, args={args}, id={t_id}")

        tool_fn = TOOLS.get(name)
        if tool_fn is None:
            p_err(f"⛔ Tool '{name}' không tồn tại!")
            result_payload = {"ok": False, "error": f"Tool '{name}' not found"}
        else:
            try:
                # thực thi tool thật
                result = tool_fn.invoke(args)
                result_payload = {"ok": True, "result": result}
                p_tool(f"↳ Result: {result}")
            except Exception as e:
                p_err(f"⛔ Tool '{name}' error: {e}")
                result_payload = {"ok": False, "error": str(e)}

        tool_msgs.append(ToolMessage(content=json.dumps(result_payload, ensure_ascii=False), tool_call_id=t_id))

    # Ghép hội thoại + AIMessage + ToolMessage để LLM tổng hợp cuối
    messages2 = messages + [ai1] + tool_msgs

    p_sys("\n=== STEP 3: LLM SYNTHESIS ===")
    ai2: AIMessage = bound.invoke(messages2)
    p_ai("AI (lần 2) → Final answer:")
    print(BOLD + ai2.content + RESET)
    return ai2.content

# ====== 4) Thử 2 ca: (A) không cần tool, (B) cần tool ======
print("\n" + "="*80)
print("CASE A: Câu xã giao (không cần tool)")
print("="*80)
run_with_tools_verbose(llm, [get_exchange_rate], "Hỏi: Bạn khỏe không?")

print("\n" + "="*80)
print("CASE B: Cần gọi tool (tỷ giá)")
print("="*80)
run_with_tools_verbose(llm, [get_exchange_rate], "Hỏi: Tỷ giá USD/VND hiện tại là bao nhiêu?")



CASE A: Câu xã giao (không cần tool)
=== STEP 1: USER → LLM ===
Human: Hỏi: Bạn khỏe không?
AI (lần 1): content = 'Tôi không có cảm xúc, nhưng cảm ơn bạn đã hỏi! Bạn cần giúp gì hôm nay?'
AI (lần 1): tool_calls = []
→ Không cần tool. Final answer:
Tôi không có cảm xúc, nhưng cảm ơn bạn đã hỏi! Bạn cần giúp gì hôm nay?

CASE B: Cần gọi tool (tỷ giá)
=== STEP 1: USER → LLM ===
Human: Hỏi: Tỷ giá USD/VND hiện tại là bao nhiêu?
AI (lần 1): content = ''
AI (lần 1): tool_calls = [{"name": "get_exchange_rate", "args": {"pair": "USD/VND"}, "id": "call_CaUVEvhxFzT4NUH1YdVPe7WZ", "type": "tool_call"}]

=== STEP 2: EXECUTE TOOLS ===
[ToolCall 1] name=get_exchange_rate, args={'pair': 'USD/VND'}, id=call_CaUVEvhxFzT4NUH1YdVPe7WZ
↳ Result: 24500

=== STEP 3: LLM SYNTHESIS ===
AI (lần 2) → Final answer:
Tỷ giá USD/VND hiện tại là 24,500 VND.


'Tỷ giá USD/VND hiện tại là 24,500 VND.'

In [15]:
# =============================
# Phase 2 – Tool Calling with Step-by-Step Chat Log
# =============================
# pip install -U langchain langchain-openai

import json
from typing import List, Dict, Any
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool

# ==== Màu in console (ANSI) ====
R="\x1b[0m"; B="\x1b[1m"
C="\x1b[36m"; G="\x1b[32m"; M="\x1b[35m"; Y="\x1b[33m"; Rr="\x1b[31m"; K="\x1b[90m"
def log_h(s): print(B+G+f"Human: {s}"+R)
def log_ai(s, tag="AI"): print(B+M+f"{tag}: {s}"+R)
def log_tool(s): print(B+Y+f"Tool: {s}"+R)
def log_sys(s): print(B+C+f"{s}"+R)
def log_dbg(s): print(K+s+R)
def log_err(s): print(B+Rr+s+R)

# ==== LLM ====
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ==== Tools (có thể bind 1 hoặc nhiều) ====
@tool
def get_exchange_rate(pair: str) -> str:
    """Tra cứu tỷ giá cơ bản (demo). pair ví dụ: 'USD/VND'."""
    rates = {"USD/VND": 24500, "EUR/VND": 26600}
    return str(rates.get(pair.upper(), "Unknown"))

@tool
def add(a: int, b: int) -> int:
    """Cộng 2 số nguyên."""
    return a + b

TOOL_MAP = {t.name: t for t in [get_exchange_rate, add]}

def run_chat_with_logs(prompt: str, tools: List):
    """
    In log hội thoại theo từng dòng:
    1) Human → LLM
    2) AI#1 (có/không tool_calls)
    3) Tool (thực thi thật + in kết quả)
    4) AI#2 Final (tổng hợp trả lời)
    """
    bound = llm.bind_tools(tools)

    # 1) Human → LLM
    messages = [
        SystemMessage(content="Bạn là trợ lý biết gọi tool chính xác, trả lời ngắn gọn."),
        HumanMessage(content=prompt),
    ]
    log_sys("=== STEP 1: USER → LLM ===")
    log_h(prompt)

    # 2) AI#1
    ai1: AIMessage = bound.invoke(messages)
    log_ai(f"(lần 1) content = {repr(ai1.content)}", "AI#1")
    log_dbg(f"AI#1.tool_calls = {json.dumps(ai1.tool_calls or [], ensure_ascii=False)}")

    # Nếu không có tool_calls → final luôn
    if not ai1.tool_calls:
        log_sys("\n=== FINAL (không cần tool) ===")
        log_ai(ai1.content, "AI#Final")
        return ai1.content

    # 3) Tool execution (có thể nhiều tool_calls)
    log_sys("\n=== STEP 2: EXECUTE TOOLS ===")
    tool_msgs: List[ToolMessage] = []
    for i, call in enumerate(ai1.tool_calls, start=1):
        name = call["name"]
        args = call.get("args", {}) or {}
        call_id = call["id"]
        log_tool(f"[{i}] {name} args={args} id={call_id}")

        tool_fn = TOOL_MAP.get(name)
        if not tool_fn:
            payload = {"ok": False, "error": f"Unknown tool '{name}'"}
            log_err(f"⛔ Unknown tool: {name}")
        else:
            try:
                result = tool_fn.invoke(args)
                payload = {"ok": True, "result": result}
                log_tool(f"↳ result = {result}")
            except Exception as e:
                payload = {"ok": False, "error": str(e)}
                log_err(f"⛔ Tool error: {e}")

        tool_msgs.append(
            ToolMessage(content=json.dumps(payload, ensure_ascii=False), tool_call_id=call_id)
        )

    # 4) AI#2 Final synthesis
    log_sys("\n=== STEP 3: LLM SYNTHESIS ===")
    final: AIMessage = bound.invoke(messages + [ai1] + tool_msgs)
    log_ai(final.content, "AI#Final")
    return final.content

# =============================
# DEMO A: Câu xã giao (không cần tool)
# =============================
print("\n" + "="*80)
print("DEMO A — Không cần tool")
print("="*80)
run_chat_with_logs("Chào bạn, hôm nay thế nào?", [get_exchange_rate, add])

# =============================
# DEMO B: Cần tool (gọi 1 tool)
# =============================
print("\n" + "="*80)
print("DEMO B — Cần tool: tỷ giá")
print("="*80)
run_chat_with_logs("Cho tôi biết tỷ giá USD/VND hiện tại.", [get_exchange_rate, add])

# =============================
# DEMO C: Nhiều tool_calls trong 1 lượt
# (LLM có thể quyết định gọi 2 tool: tính 68+32 và hỏi thêm tỷ giá)
# =============================
print("\n" + "="*80)
print("DEMO C — Gọi nhiều tool: add + get_exchange_rate")
print("="*80)
run_chat_with_logs("Tính 68+32 và cho tôi tỷ giá EUR/VND luôn nhé.", [get_exchange_rate, add])



DEMO A — Không cần tool
=== STEP 1: USER → LLM ===
Human: Chào bạn, hôm nay thế nào?
AI#1: (lần 1) content = 'Chào bạn! Mình luôn sẵn sàng giúp đỡ bạn. Bạn cần hỗ trợ gì hôm nay?'
AI#1.tool_calls = []

=== FINAL (không cần tool) ===
AI#Final: Chào bạn! Mình luôn sẵn sàng giúp đỡ bạn. Bạn cần hỗ trợ gì hôm nay?

DEMO B — Cần tool: tỷ giá
=== STEP 1: USER → LLM ===
Human: Cho tôi biết tỷ giá USD/VND hiện tại.
AI#1: (lần 1) content = ''
AI#1.tool_calls = [{"name": "get_exchange_rate", "args": {"pair": "USD/VND"}, "id": "call_YbbCkaDKi7RmR2vRc8MMP0Xu", "type": "tool_call"}]

=== STEP 2: EXECUTE TOOLS ===
Tool: [1] get_exchange_rate args={'pair': 'USD/VND'} id=call_YbbCkaDKi7RmR2vRc8MMP0Xu
Tool: ↳ result = 24500

=== STEP 3: LLM SYNTHESIS ===
AI#Final: Tỷ giá USD/VND hiện tại là 24,500 VND.

DEMO C — Gọi nhiều tool: add + get_exchange_rate
=== STEP 1: USER → LLM ===
Human: Tính 68+32 và cho tôi tỷ giá EUR/VND luôn nhé.
AI#1: (lần 1) content = ''
AI#1.tool_calls = [{"name": "add", "args": {

'Kết quả của 68 + 32 là 100. Tỷ giá EUR/VND là 26,600.'

In [ ]:
┌────────────────────┐
│   Human (prompt)   │
└─────────┬──────────┘
          │
          ▼
┌────────────────────┐
│   LLM (bind_tools) │
│  - quyết định call │
│  - tạo tool_calls  │
└─────────┬──────────┘
          │ AIMessage(tool_calls=[{name,args,id}, ...])
          ▼
┌────────────────────────────────────────────┐
│          Tool Orchestrator (Python)        │
│  for each call in tool_calls:              │
│   1) tìm tool theo name (TOOL_MAP[name])   │
│   2) tool.invoke(args: dict)               │
│       ┌─────────────────────────────────┐  │
│       │   BaseTool/StructuredTool       │  │
│       │  - args_schema (Pydantic Model) │  │
│       │  - parse: model(**args)         │  │
│       │  - func(**parsed.dict())        │  │
│       └─────────────────────────────────┘  │
│   3) nhận result → ToolMessage(tool_id)    │
└─────────┬──────────────────────────────────┘
          │ (append AIMessage + ToolMessage)
          ▼
┌────────────────────┐
│  LLM (lần 2)       │
│  - đọc ToolMessage │
│  - tổng hợp answer │
└─────────┬──────────┘
          │
          ▼
┌────────────────────┐
│  Final answer      │
└────────────────────┘


In [ ]:
BaseTool (abstract)
├─ Tool
│   └─ (func: Callable[[str], Any], input: str | Any)
│
├─ StructuredTool
│   └─ (func: Callable, args_schema: Pydantic Model, input: dict)
│
└─ DynamicStructuredTool
    └─ (func: Callable, input: dict[Any], schema nới lỏng)
